In [ ]:
import numpy as np

from fpcross import ij, Grid, Solver, Model, Check

ij()

Let solve
$$
    \frac{\partial \psi}{\partial t}
    =
    - A \psi,
    \quad
    \psi(0) = \psi_0,
    \quad
    x \in \Omega,
$$
where
$$
    A \psi
    =
    - \epsilon \Delta \psi
    + div({\psi v}),
$$
with $\epsilon \in R$ and $v \in R^{d}$.

Note that for the case of potential field $v = grad(\phi)$ the analytical solution of the stationary problem is given as $\psi = C e^{-\frac{\phi}{\epsilon}}$, where the constant $C$ is defined to satisfy the normalization condition $\int_{\Omega} \psi \, d x = 1$.

Consider three-dimensional equation (dumbbell model)
$$
    \epsilon=\frac{1}{2},
    \quad
    v = K x - \frac{1}{2} grad(\phi),
    \quad
    x \in [-a, a]^3,
$$
where
$$
    \phi = \frac{1}{2} x^2 + \frac{\alpha}{p^3} e^{-\frac{x^2}{2 p^2}},
$$
and
$$
    K = \beta \begin{bmatrix}
        0 & 1 & 0 \\
        0 & 0 & 0 \\
        0 & 0 & 0
    \end{bmatrix}.
$$

We have
$$
    v
    =
    K x
    - \frac{1}{2} x
    + \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}} x,
$$
$$
    \frac{d v}{d x}
    =
    K
    - \frac{1}{2} I
    + \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}}
    - \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}} x,
$$

In [ ]:
s = 1.
D = 0.5

class Model_(Model):

    def d(self):
        ''' Spatial dimension. '''
        return 3
    
    def D(self):
        ''' Diffusion coefficient. '''
        return D

    def f0(self, X, t):
        ''' Function f(x, t). '''
        return np.zeros(X.shape)

    def f1(self, X, t):
        ''' Function f(x, t) / d x. '''
        return np.zeros(X.shape)

    def r0(self, X):
        ''' Initial condition. '''
        a = 2. * s
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)

    def rt(self, X, t):
        ''' Exact analytic solution. '''
        a = 2. * s + 4. * D * t
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)
    
    def with_rt(self):
        return True

In [ ]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+0., +1.], kind='u'), # Time grid
    SG=Grid(d=3, n=21, l=[-5., +5.], kind='c'), # Spatial grid
    MD=Model_()                                 # Model for equation
)
SL.init()    # Init solver
SL.prep()    # Prepare special matrices
SL.calc()    # Solve equation
SL.TG.info() # Present results from all submodules
SL.SG.info()
SL.FN.info()
SL.info()